# Main testing

In [ ]:
import gym
import minihack
import numpy as np
import math
import matplotlib.pyplot as plt
import IPython.display as display
from typing import Tuple, List
from utils import *
from algorithms import *
from logic import *

## Creation of the environment

In [ ]:
env = gym.make("MiniHack-River-Narrow-v0", observation_keys=("chars", "pixel"))
state = env.reset()
env.render()

Printing the structure of the map and the graphic visualization 

In [ ]:
state['chars'].shape, state['pixel'].shape

In [ ]:
plt.imshow(state['pixel'][100:250, 400:750]) #Immagine ristretta con range [y][x]

In [ ]:
game_map = state['chars']
game = state['pixel']

## Finding coordinates
Searching for the coordinates of the player, all the boulders and river

In [ ]:
start = get_player_location(game_map)
boulders = get_boulder_locations(game_map)
print("Agent position:", start)


Coordinates of each block present in the map

In [ ]:
print("Boulders positions: ", boulders)

Calculating the distance to the shortest boulder, which may be not the best to push becouse doesn't consider also the distance to the river

In [ ]:
target_block_position, distance = get_min_distance_point_to_points(start[0],start[1],boulders)
print("Target position:", target_block_position, "Distance:", distance)

### Finding river positions

In [ ]:
river_positions = get_river_locations(game_map)
print("River positions: ", river_positions)

## Complete path player - boulder - river

1) A star calculates the path from the player to the optimal boulder to push
2) A star calculates the path from the boulder reached to the nearest river position

## Path boulder to river

Calculating the coordinates of the best boulder to push

In [ ]:
coordinates_min_boulder = get_best_global_distance(start, boulders, river_positions)
print("Coordinates of the best boulder to push: ",coordinates_min_boulder)
#pushing_position = position_for_boulder_push(coordinates_min_boulder, )

## Path Boulder -> River
Calculating the path from the boulder to the river shortest distance

In [ ]:
temp = get_min_distance_point_to_points(coordinates_min_boulder[0],coordinates_min_boulder[1], river_positions)
final_position = tuple(temp[0])

print("Boulders coordinates:",coordinates_min_boulder,"River position: ", final_position)

path_boulder_river = a_star(game,game_map, coordinates_min_boulder, final_position, get_optimal_distance_point_to_point)

path_boulder_river.append(final_position) #Adding the river final position, otherwise will drop the boulder near the river but not in the river

print("Path from boulder to river: ",path_boulder_river)

Calculating the position in which the agent have to be in order to push correctly the boulder into the river

In [ ]:
pushing_position = position_for_boulder_push(coordinates_min_boulder, path_boulder_river[1])[1]#path_boulder_river[1] is the position of the second step of the boulder
print("Pushing position: ", pushing_position)

## Path Player -> Boulder pushing position

In [ ]:
path_player_to_pushing_position = a_star(game,game_map, start, pushing_position, get_optimal_distance_point_to_point)
#path_player_to_pushing_position.append(pushing_position) #append the pushing position otherwise is near the correct cell and not in the cell
print("Path from the start to the pushing position: ",path_player_to_pushing_position)

Given the path of the boulder to the river we generated the path that the agent has to do in order to "stay attached" to the boulder and pushing it into the river

In [ ]:

agent_actions,path_player_to_river = push_boulder_path(path_boulder_river)
print("Agent path: ", path_player_to_river)


Merging the two path together

In [ ]:
agent_full_path = path_player_to_pushing_position + path_player_to_river
print("Full path of the agent: ",agent_full_path)

## Visualization of the execution
Showing the animated path from start to finish

In [ ]:
actions, names = actions_from_path(start, agent_full_path) 
player_pos = plot_animated_sequence(env,game, game_map,actions[1:]) #Remove first action because it is the start position
